In [1]:
from data.mnist_loader import load_mnist_batches
from evaluation.evaluation import evaluate_outputs
from models.single_trainable import Trainable
from training.single_model_trainer import Trainer
from utils.globals import MAX_EPOCHS
from utils.state import State

In [2]:
NUM_INSTANCES = 5

In [3]:
def run_normal_single():
    state = State(federated=False, neuromorphic=False, method='backprop', save_model=True)

    batches_mnist_dataset = load_mnist_batches()

    trainable = Trainable(state=state)
    trainer = Trainer(trainable=trainable, dataset=batches_mnist_dataset, state=state)


    trainer.train_model()

    metrics = evaluate_outputs(trainable.model, batches_mnist_dataset.test_loader)

    final_metrics = metrics.get_results()
    return final_metrics, trainer.training_scores

In [4]:
all_training_scores = []
all_final_metrics = []

In [ ]:
for i in range(NUM_INSTANCES):
    final_metrics, training_scores = run_normal_single()
    all_final_metrics.append(final_metrics)
    all_training_scores.append(training_scores)

Testing: 100%|██████████| 79/79 [00:03<00:00, 23.15it/s, Accuracy=98.76%]


In [7]:
all_training_scores

[{'Training Loss': [1.3796706365493305,
   0.1296316057161281,
   0.08877082426297037,
   0.07343834551018581],
  'Training Accuracy': [72.56315789473685,
   96.05087719298245,
   97.26140350877193,
   97.78947368421052],
  'Validation Loss': [0.17390754016240437,
   0.08504797240098318,
   0.06534145454565685,
   0.06622807167967161],
  'Validation Accuracy': [94.8, 97.53333333333333, 98.13333333333334, 97.8]},
 {'Training Loss': [1.2089910922217788,
   0.12113825597679406,
   0.08503129948126642,
   0.06864806332086262],
  'Training Accuracy': [79.15964912280701,
   96.4140350877193,
   97.3719298245614,
   97.91929824561403],
  'Validation Loss': [0.15055502518018088,
   0.08988835259278616,
   0.06260179362694422,
   0.062049742539723715],
  'Validation Accuracy': [95.5,
   97.46666666666667,
   98.03333333333333,
   98.13333333333334]},
 {'Training Loss': [1.0343937897326654,
   0.11984510836528059,
   0.0846940140515043,
   0.06932713890284822],
  'Training Accuracy': [79.2122807

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
# Compute means and standard deviations

In [ ]:
# Enhanced plotting: Showing individual lines for each run and clearer visualization of mean with std
dfc = plt.rcParams['axes.prop_cycle'].by_key()['color']
c1 = dfc[0]
c2 = dfc[1]


In [21]:
cols = {'train_mean': dfc[0], 'train_runs': dfc[0], 'val_mean': dfc[1], 'val_runs':dfc[1]}


def plot_loss_with_mean_std(data, colors=cols):
    epochs = MAX_EPOCHS
    assert epochs == len(data[0]['Training Loss'])

    """
    Plot Loss with individual runs, mean, and std as shaded area.

    Args:
        data: List of dictionaries containing Training Loss and Validation Loss data.
        colors: Dictionary to specify colors for mean, runs, and std for training and validation.

    Returns:
        None
    """
    mean_training_loss = np.mean([d['Training Loss'] for d in data], axis=0)
    std_training_loss = np.std([d['Training Loss'] for d in data], axis=0)
    
    mean_validation_loss = np.mean([d['Validation Loss'] for d in data], axis=0)
    std_validation_loss = np.std([d['Validation Loss'] for d in data], axis=0)
    
    # Plot Loss
    plt.figure(figsize=(12, 6))
    for run in data:
        plt.plot(range(1, epochs + 1), run['Training Loss'], '--', color=colors['train_runs'], alpha=0.3)
        plt.plot(range(1, epochs + 1), run['Validation Loss'], '--', color=colors['val_runs'], alpha=0.3)

    plt.plot(range(1, epochs + 1), mean_training_loss, '-', color=colors['train_mean'], label='Training Loss')
    plt.fill_between(range(1, epochs + 1), 
                     mean_training_loss - std_training_loss, 
                     mean_training_loss + std_training_loss, 
                     color=colors['train_mean'], alpha=0.2)

    plt.plot(range(1, epochs + 1), mean_validation_loss, '-', color=colors['val_mean'], label='Validation Loss')
    plt.fill_between(range(1, epochs + 1), 
                     mean_validation_loss - std_validation_loss, 
                     mean_validation_loss + std_validation_loss, 
                     color=colors['val_mean'], alpha=0.2)

    plt.title('Loss Over Epochs (Individual Runs and Mean with Std)')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()